In [1]:
import os 
from tqdm import tqdm
import torchvision
import numpy as np

In [2]:
names = []
for root, dirs, files in os.walk(os.path.join("..", "data","lenses"), topdown=False):
   for name in files:
      names.append(os.path.join(root, name))

In [3]:
print(names[0])
print(len(names))

..\data\lenses\no_sub\image_100108096845777232664144315147734473464.jpg
10000


In [4]:
print(len(names))

10000


In [13]:
def calculate_mean_std_2pass_low_memory(directory):
    names = []
    for root, dirs, files in os.walk(directory, topdown=False):
        for name in files:
            names.append(os.path.join(root, name))

    minima = float("inf")
    maxima = float("-inf")
    counts = 0
    means = 0
    for f in tqdm(names):
        try:
            mat = torchvision.io.read_image(f).numpy() / 255.
            mat = mat.reshape(mat.shape[1], mat.shape[2])
            v=  mat.shape[0] * mat.shape[1]
            counts += np.array(v)
            means += np.sum(mat.reshape(-1), axis=0)
            minima = min(minima, np.min(mat.reshape(-1)))
            maxima = max(maxima, np.max(mat.reshape(-1)))
            
        except ValueError:
            pass

    means = means / counts

    stds = 0
    for f in tqdm(names):
        try:
            mat = torchvision.io.read_image(f).numpy() / 255.
            stds += np.sum( (mat.reshape(-1) - means)**2, axis=0)
        except ValueError:
            pass
    stds = np.sqrt( stds / counts ) 

    print(f"minima in data : {minima}, maxima in data {maxima}")
    return means, stds

calculate_mean_std_2pass_low_memory(os.path.join("..","data", "lenses"))

100%|██████████| 10000/10000 [00:04<00:00, 2156.43it/s]

minima in data : 0.0, maxima in data 1.0


(0.022538540984749413, 0.06030143571638838)